In [40]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import networkx as nx

import torch_geometric as tg
from torch_geometric.utils import to_networkx
from torch_geometric.utils import degree
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from torch_geometric.datasets import Planetoid

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Datasets

## Cora

In [42]:
cora_dataset = Planetoid(root='./data', name='Cora')

data = cora_dataset[0]

print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [43]:
num_nodes = data.num_nodes
print(f"No. of nodes: {num_nodes}")

num_edges = data.num_edges
print(f"No. of edges: {num_edges}")

num_features = data.num_features
print(f"No. of features: {num_features}")

num_classes = len(set(data.y.numpy()))
print(f"No. of classes: {num_classes}")

train_mask = data.train_mask
print(f"No. of nodes in training set: {train_mask.sum().item()}")

val_mask = data.val_mask
print(f"No. of nodes in validation set: {val_mask.sum().item()}")

test_mask = data.test_mask
print(f"No. of nodes in test set: {test_mask.sum().item()}")

No. of nodes: 2708
No. of edges: 10556
No. of features: 1433
No. of classes: 7
No. of nodes in training set: 140
No. of nodes in validation set: 500
No. of nodes in test set: 1000


In [46]:
train_data = data[train_mask]
val_data = data[val_mask]
test_data = data[test_mask]

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

TypeError: unhashable type: 'numpy.ndarray'

# Models

## GCN

In [24]:
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [25]:
model = GCN().to(device)
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


# Training

In [36]:
def train(model, train_loader, optimizer):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x.to(device), batch.edge_index.to(device))
        loss = F.nll_loss(out, batch.y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def test(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        with torch.no_grad():
            pred = model(data.x.to(device), data.edge_index.to(device)).max(dim=1)[1]
        correct += int((pred == data.y.to(device)).sum())
    return correct / num_nodes

In [37]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 200


for epoch in range(1, epochs+1):
    loss = train(model, train_loader, optimizer)
    acc = test(model, cora_dataset)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: {acc:.4f}')

Epoch: 001, Loss: 0.1160, Acc: 0.9860
Epoch: 002, Loss: 0.1137, Acc: 0.9871
Epoch: 003, Loss: 0.1160, Acc: 0.9852
Epoch: 004, Loss: 0.1064, Acc: 0.9808
Epoch: 005, Loss: 0.1017, Acc: 0.9808
Epoch: 006, Loss: 0.1170, Acc: 0.9819
Epoch: 007, Loss: 0.1057, Acc: 0.9830
Epoch: 008, Loss: 0.1114, Acc: 0.9845
Epoch: 009, Loss: 0.1090, Acc: 0.9849
Epoch: 010, Loss: 0.1058, Acc: 0.9860
Epoch: 011, Loss: 0.1084, Acc: 0.9849
Epoch: 012, Loss: 0.1080, Acc: 0.9849
Epoch: 013, Loss: 0.1071, Acc: 0.9841
Epoch: 014, Loss: 0.0964, Acc: 0.9845
Epoch: 015, Loss: 0.1006, Acc: 0.9860
Epoch: 016, Loss: 0.1021, Acc: 0.9860
Epoch: 017, Loss: 0.0974, Acc: 0.9871
Epoch: 018, Loss: 0.0987, Acc: 0.9867
Epoch: 019, Loss: 0.1032, Acc: 0.9860
Epoch: 020, Loss: 0.1062, Acc: 0.9860
Epoch: 021, Loss: 0.1071, Acc: 0.9863
Epoch: 022, Loss: 0.0976, Acc: 0.9863
Epoch: 023, Loss: 0.0996, Acc: 0.9878
Epoch: 024, Loss: 0.1017, Acc: 0.9874
Epoch: 025, Loss: 0.1037, Acc: 0.9874
Epoch: 026, Loss: 0.0913, Acc: 0.9874
Epoch: 027, 